# Fenwick Tree

![Fenwick tree](fenwick.png)

If can precompute total sum of elements before specific point in the sequence, then we can get cumulative sum up to a point by just summing up some specific precomputed sums.

Fenwick tree can be used with different configurations:
- Point updates and Range queries
- Range updates and Point queries
- Range updates and Range queries

## Point updates and Range queries

In [10]:
class FenwickTree:
    
    def __init__(self, max_capacity):
        self.data = [0] * (max_capacity + 1)
    
    def __len__(self):
        return len(self.data) - 1

    def __getitem__(self, index):
        return self._query(index)
    
    def update(self, index, value):
        index += 1
        while index < len(self.data):
            self.data[index] += value
            index += index & (-index)
    
    def _query(self, end):
        result = 0
        i = end + 1
        while i > 0:
            result += self.data[i]
            i -= i & (-i)
        return result
    
    def sum(self, end, start=0):
        """Get sum from start to end, inclusive"""
        result = self._query(end)
        if start > 0:
            result -= self._query(start - 1)
        return result

In [11]:
tree = FenwickTree(10)
tree.update(4, 3)
tree.update(2, 4)
print('sum from 0 up to 5 =', tree.sum(5))
print('sum from 6 up to 8 =', tree.sum(start=6, end=8))

sum from 0 up to 5 = 7
sum from 6 up to 8 = 0


## Range updates and Point queries

In [33]:
class FenwickTree:
    
    def __init__(self, max_capacity):
        self.data = [0] * (max_capacity + 1)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.query(index)
    
    def _update(self, index, value):
        index += 1
        while index < len(self.data):
            self.data[index] += value
            index += index & (-index)
    
    def update(self, start, end, value):
        self._update(start, value)
        self._update(end + 1, -value)
    
    def query(self, end):
        result = 0
        i = end + 1
        while i > 0:
            result += self.data[i]
            i -= i & (-i)
        return result

In [37]:
tree = FenwickTree(10)
tree.update(3, 7, 3)
tree.update(5, 8, 4)
tree.query(6)

7

## Range updates and Range queries

**Case A**: $k < l$

![Case A](case_a.png)

$$sum = 0 $$

** Case B**: $l \leq k \leq r$

![Case B](case_b.png)

$$ sum = val*k - val*(l-1) $$

**Case C**: $r < k < n$

![Case C](case_c.png)

$$\begin{eqnarray} sum &=& val*r - val*(l-1) \end{eqnarray}$$

In [55]:
class FenwickTree:
    
    def __init__(self, max_capacity):
        self.data = [0] * (max_capacity + 1)
        self.fix_factors = [0] * (max_capacity + 1)
    
    def __len__(self):
        return len(self.data) - 1
    
    def __getitem__(self, index):
        return 0
    
    def _update(self, tree, index, value):
        index += 1
        while index < len(tree):
            tree[index] += value
            index += index & (-index)
    
    def update(self, start, end, value):
        self._update(self.data, start, value)
        self._update(self.data, end + 1, -value)
        self._update(self.fix_factors, start, value * (start + 1 - 1))
        self._update(self.fix_factors, end + 1, -value * (end + 1))
    
    def _query(self, tree, end):
        result = 0
        i = end + 1
        while i > 0:
            result += tree[i]
            i -= i & (-i)
        return result
    
    def _query_cum(self, end):
        k = end + 1
        return (self._query(self.data, k) * k -
                self._query(self.fix_factors, k))
    
    def query(self, end, start=0):
        return self._query_cum(end) - self._query_cum(start - 1)
        

In [66]:
tree = FenwickTree(10)
tree.update(3, 7, 2)
tree.update(4, 5, 3)
tree.query(8, start=5)

9